# Exporting data from BigQuery to Google Cloud Storage

In this notebook, we export BigQuery data to GCS so that we can reuse our Keras model that was developed on CSV data.

https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/art_and_science_of_ml/solutions/export_data_from_bq_to_gcs.ipynb

# Run the chown command to change the ownership of the repository
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

# Install the Google Cloud BigQuery library
%pip install google-cloud-bigquery==1.25.0

Please ignore any incompatibility warnings and errors.
**Restart** the kernel to use updated packages. (On the Notebook menu, select Kernel > Restart Kernel > Restart).


In [7]:
# The OS module in python provides functions for interacting with the operating system.
import os

from google.cloud import bigquery

Change the following cell as necessary:

In [8]:
!gsutil mb -l europe-west1 gs://vertex_e2e_taxi_data

Creating gs://vertex_e2e_taxi_data/...
ServiceException: 409 A Cloud Storage bucket named 'vertex_e2e_taxi_data' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [9]:
bucket = 'vertex_e2e_taxi_data'

In [10]:
# Change with your own bucket and project below:
BUCKET =  bucket
PROJECT = "vf-grp-commercial-tst-explore"

OUTDIR = "gs://{bucket}/taxifare/data".format(bucket=BUCKET)

os.environ['BUCKET'] = BUCKET
os.environ['OUTDIR'] = OUTDIR
os.environ['PROJECT'] = PROJECT

## Create BigQuery tables

If you haven not already created a BigQuery dataset for our data, run the following cell:

In [6]:
# Initialize a BigQuery client.
bq = bigquery.Client(project = PROJECT)
dataset = bigquery.Dataset(bq.dataset("taxifare"))

# Create a new dataset with the `create_dataset()` method.
try:
    bq.create_dataset(dataset)
    print("Dataset created")
except:
    print("Dataset already exists")

Dataset created


Let's create a table with 1 million examples.

Note that the order of columns is exactly what was in our CSV files.

In [7]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_training_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `bigquery-public-data.new_york.tlc_yellow_trips_2016`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) between 0 and 10
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 986.74query/s]                         


""


Make the validation dataset be 1/10 the size of the training dataset.

In [8]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_valid_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `bigquery-public-data.new_york.tlc_yellow_trips_2016`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 11
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1491.75query/s]                        


""


create test data, as well as prediction data (with headers, without label)

In [1]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_test_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `bigquery-public-data.new_york.tlc_yellow_trips_2016`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 12
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 1422.44query/s]                        


""


In [3]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_pred_data AS

SELECT
    #(tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `bigquery-public-data.new_york.tlc_yellow_trips_2016`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 13
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 884.00query/s]                         


""


## Export the tables as CSV files

## bq location us. can this be restricted to eu?

In [11]:
%%bash

echo "Deleting current contents of $OUTDIR"
gsutil -m -q rm -rf $OUTDIR

echo "Extracting training data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_training_data \
   $OUTDIR/taxi-train-*.csv

echo "Extracting validation data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_valid_data \
   $OUTDIR/taxi-valid-*.csv

echo "Extracting test data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_test_data \
   $OUTDIR/taxi-test-*.csv

echo "Extracting prediction data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_pred_data \
   $OUTDIR/taxi-pred-*.csv

# With `-l` option, gsutil will output additional information about each matching provider, bucket, subdirectory, or object.
gsutil ls -l $OUTDIR

Deleting current contents of gs://vertex_e2e_taxi_data/taxifare/data
Extracting training data to gs://vertex_e2e_taxi_data/taxifare/data
Extracting validation data to gs://vertex_e2e_taxi_data/taxifare/data
Extracting test data to gs://vertex_e2e_taxi_data/taxifare/data
Extracting prediction data to gs://vertex_e2e_taxi_data/taxifare/data
   7549354  2021-11-11T17:16:13Z  gs://vertex_e2e_taxi_data/taxifare/data/taxi-pred-000000000000.csv
   7789443  2021-11-11T17:16:05Z  gs://vertex_e2e_taxi_data/taxifare/data/taxi-test-000000000000.csv
  84774097  2021-11-11T17:15:43Z  gs://vertex_e2e_taxi_data/taxifare/data/taxi-train-000000000000.csv
   7549354  2021-11-11T17:15:57Z  gs://vertex_e2e_taxi_data/taxifare/data/taxi-valid-000000000000.csv
TOTAL: 4 objects, 107662248 bytes (102.67 MiB)


Waiting on bqjob_r794a1168b7d2b5bc_0000017d0fff4a13_1 ... (22s) Current status: DONE   
Waiting on bqjob_r6477e5104649498_0000017d0fffb06d_1 ... (3s) Current status: DONE   
Waiting on bqjob_r177dca2e9d0724de_0000017d0fffcaa4_1 ... (4s) Current status: DONE   
Waiting on bqjob_r19bd74ecf5e061bc_0000017d0fffe962_1 ... (4s) Current status: DONE   


In [13]:
# The cat command outputs the contents of one or more URLs to stdout.
!gsutil cat gs://$BUCKET/taxifare/data/taxi-train-000000000000.csv | head -2

2.5,2016-04-22 12:20:52 UTC,-73.973358154296875,40.752120971679688,-73.97332763671875,40.752189636230469,2,unused
2.5,2016-05-31 20:54:22 UTC,-73.967742919921875,40.765071868896484,-73.967803955078125,40.765121459960938,2,unused


In [15]:
!gsutil cat gs://$BUCKET/taxifare/data/taxi-train-000000000000.csv |wc

 749732 2249196 84774097


In [16]:
!gsutil cat gs://$BUCKET/taxifare/data/taxi-valid-000000000000.csv |wc

  67931  203793 7682062


Copyright 2020 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.